In [15]:
import json
from pathlib import Path
import sys

# Add src to path to import utils
sys.path.insert(0, str(Path.cwd().parent / "src"))

from utils.evaluate_mcq_order import load_examples
from utils.mcq_order_models import MCQOrderExample

# Load first 3 examples from the benchmark
dataset_path = Path("../data/mcq_event_timeline_strong.jsonl")
audio_root = Path("../data/audio")

examples = load_examples(dataset_path, limit=10)

print(f"Loaded {len(examples)} examples\n")
print("=" * 80)

Loaded 10 examples



In [16]:
# Display raw temporal data (onset, offset, duration, etc.) for each example
print("Raw Temporal Data from Base Dataset\n")
print("=" * 80)

for i, example in enumerate(examples, 1):
    raw_data = example.raw
    base_event = raw_data.get("base_event", {})
    
    print(f"\n{'='*80}")
    print(f"Example {i}: {example.example_id}")
    print(f"Audio: {example.audio_filename}")
    print(f"{'='*80}")
    
    # Display base event temporal information
    print(f"\n📍 Base Event (the event being asked about):")
    print(f"   Text: \"{base_event.get('text', 'N/A')}\"")
    print(f"   Event Index: {base_event.get('event_index', 'N/A')}")
    print(f"   Onset: {base_event.get('onset', 'N/A'):.3f}s" if isinstance(base_event.get('onset'), (int, float)) else f"   Onset: {base_event.get('onset', 'N/A')}")
    print(f"   Offset: {base_event.get('offset', 'N/A'):.3f}s" if isinstance(base_event.get('offset'), (int, float)) else f"   Offset: {base_event.get('offset', 'N/A')}")
    print(f"   Duration: {base_event.get('duration', 'N/A'):.3f}s" if isinstance(base_event.get('duration'), (int, float)) else f"   Duration: {base_event.get('duration', 'N/A')}")
    print(f"   Occurrence Count: {base_event.get('occurrence_count', 'N/A')}")
    print(f"   Last Offset: {base_event.get('last_offset', 'N/A'):.3f}s" if isinstance(base_event.get('last_offset'), (int, float)) else f"   Last Offset: {base_event.get('last_offset', 'N/A')}")
    
    # Display options with temporal information
    print(f"\n📋 Options with Temporal Information:")
    for option in example.options:
        option_raw = None
        # Find matching option in raw data
        for opt in raw_data.get("options", []):
            if opt.get("label") == option.label:
                option_raw = opt
                break
        
        print(f"\n   {option.label}. {option.text}")
        print(f"      Type: {option.option_type}")
        
        if option_raw:
            if option.option_type == "event":
                print(f"      Event Index: {option_raw.get('event_index', 'N/A')}")
                print(f"      Onset: {option_raw.get('onset', 'N/A'):.3f}s" if isinstance(option_raw.get('onset'), (int, float)) else f"      Onset: {option_raw.get('onset', 'N/A')}")
                print(f"      Offset: {option_raw.get('offset', 'N/A'):.3f}s" if isinstance(option_raw.get('offset'), (int, float)) else f"      Offset: {option_raw.get('offset', 'N/A')}")
                print(f"      Duration: {option_raw.get('duration', 'N/A'):.3f}s" if isinstance(option_raw.get('duration'), (int, float)) else f"      Duration: {option_raw.get('duration', 'N/A')}")
                print(f"      Occurrence Count: {option_raw.get('occurrence_count', 'N/A')}")
                print(f"      Last Offset: {option_raw.get('last_offset', 'N/A'):.3f}s" if isinstance(option_raw.get('last_offset'), (int, float)) else f"      Last Offset: {option_raw.get('last_offset', 'N/A')}")
            else:
                print(f"      (No temporal data - this is a 'none' option)")
        
        if option.label == example.answer_label:
            print(f"      ✅ CORRECT ANSWER")
    
    print()

Raw Temporal Data from Base Dataset


Example 1: 100388.mp3__0
Audio: 100388.mp3

📍 Base Event (the event being asked about):
   Text: "A river flows loudly outdoors."
   Event Index: 0
   Onset: 0.016s
   Offset: 23.888s
   Duration: 23.872s
   Occurrence Count: 1
   Last Offset: 23.888s

📋 Options with Temporal Information:

   A. A bird tweets quietly and repeatedly outdoors.
      Type: event
      Event Index: 1
      Onset: 3.699s
      Offset: 4.274s
      Duration: 0.575s
      Occurrence Count: 6
      Last Offset: 23.780s
      ✅ CORRECT ANSWER

   B. This is the last event, no immediate event after.
      Type: none
      (No temporal data - this is a 'none' option)


Example 2: 100388.mp3__1
Audio: 100388.mp3

📍 Base Event (the event being asked about):
   Text: "A bird tweets quietly and repeatedly outdoors."
   Event Index: 1
   Onset: 3.699s
   Offset: 4.274s
   Duration: 0.575s
   Occurrence Count: 6
   Last Offset: 23.780s

📋 Options with Temporal Information:

   A. T

In [17]:
# Display each example with its question and audio
from IPython.display import Audio, display, Markdown

# Load Audio Flamingo 3 predictions
decisions_path = Path("../results/mcq-order/audio-flamingo-3-no-audio/20260217_090228/decisions.jsonl")
decisions_by_example_id = {}

if decisions_path.exists():
    with open(decisions_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                decision = json.loads(line)
                decisions_by_example_id[decision["example_id"]] = decision
    print(f"✅ Loaded {len(decisions_by_example_id)} Audio Flamingo 3 predictions\n")
else:
    print(f"⚠️  Decisions file not found: {decisions_path}\n")

for i, example in enumerate(examples, 1):
    print(f"\n{'='*80}")
    print(f"Example {i}: {example.example_id}")
    print(f"{'='*80}")
    
    # Display question
    print(f"\n📝 Question:")
    print(f"   {example.question}\n")
    
    # Display options
    print("📋 Options:")
    for option in example.options:
        marker = "✅" if option.label == example.answer_label else "  "
        print(f"   {marker} {option.label}. {option.text}")
    
    print(f"\n🎯 Correct Answer: {example.answer_label} - {example.answer_text}")
    
    # Display Audio Flamingo 3 prediction
    if example.example_id in decisions_by_example_id:
        decision = decisions_by_example_id[example.example_id]
        is_correct = decision["is_correct"]
        status_icon = "✅" if is_correct else "❌"
        print(f"\n🤖 Audio Flamingo 3 Prediction: {status_icon}")
        print(f"   Predicted: {decision['predicted_label']} - {decision['predicted_text']}")
        print(f"   {'Correct!' if is_correct else 'Incorrect'}")
    else:
        print(f"\n⚠️  No prediction found for this example")
    
    # Load and display audio
    audio_path = audio_root / example.audio_filename
    if audio_path.exists():
        print(f"\n🔊 Audio: {example.audio_filename}")
        display(Audio(str(audio_path)))
    else:
        print(f"\n⚠️  Audio file not found: {audio_path}")
        print(f"   (Audio directory may need to be extracted from data/audio.zip)")
    
    print()

✅ Loaded 1000 Audio Flamingo 3 predictions


Example 1: 100388.mp3__0

📝 Question:
   What happens immediately after this event first appears: "A river flows loudly outdoors."?

📋 Options:
   ✅ A. A bird tweets quietly and repeatedly outdoors.
      B. This is the last event, no immediate event after.

🎯 Correct Answer: A - A bird tweets quietly and repeatedly outdoors.

🤖 Audio Flamingo 3 Prediction: ❌
   Predicted: B - This is the last event, no immediate event after.
   Incorrect

🔊 Audio: 100388.mp3




Example 2: 100388.mp3__1

📝 Question:
   What happens immediately after this event first appears: "A bird tweets quietly and repeatedly outdoors."?

📋 Options:
   ✅ A. This is the last event, no immediate event after.
      B. A river flows loudly outdoors.

🎯 Correct Answer: A - This is the last event, no immediate event after.

🤖 Audio Flamingo 3 Prediction: ✅
   Predicted: A - This is the last event, no immediate event after.
   Correct!

🔊 Audio: 100388.mp3




Example 3: 100476.mp3__0

📝 Question:
   What happens immediately after this event first appears: "A saxophone plays a soft melody with oscillating volume."?

📋 Options:
      A. This is the last event, no immediate event after.
   ✅ B. A saxophone plays a soft melody with varying volume and fades out.
      C. A string is plucked repeatedly without pauses.

🎯 Correct Answer: B - A saxophone plays a soft melody with varying volume and fades out.

🤖 Audio Flamingo 3 Prediction: ✅
   Predicted: B - A saxophone plays a soft melody with varying volume and fades out.
   Correct!

🔊 Audio: 100476.mp3




Example 4: 100476.mp3__1

📝 Question:
   What happens immediately after this event first appears: "A saxophone plays a soft melody with varying volume and fades out."?

📋 Options:
   ✅ A. A string is plucked repeatedly without pauses.
      B. This is the last event, no immediate event after.
      C. A saxophone plays a soft melody with oscillating volume.

🎯 Correct Answer: A - A string is plucked repeatedly without pauses.

🤖 Audio Flamingo 3 Prediction: ❌
   Predicted: C - A saxophone plays a soft melody with oscillating volume.
   Incorrect

🔊 Audio: 100476.mp3




Example 5: 100476.mp3__2

📝 Question:
   What happens immediately after this event first appears: "A string is plucked repeatedly without pauses."?

📋 Options:
   ✅ A. This is the last event, no immediate event after.
      B. A saxophone plays a soft melody with oscillating volume.
      C. A saxophone plays a soft melody with varying volume and fades out.

🎯 Correct Answer: A - This is the last event, no immediate event after.

🤖 Audio Flamingo 3 Prediction: ❌
   Predicted: B - A saxophone plays a soft melody with oscillating volume.
   Incorrect

🔊 Audio: 100476.mp3




Example 6: 100481.mp3__0

📝 Question:
   What happens immediately after this event first appears: "Birds chirping in the distance."?

📋 Options:
      A. A bird chirps loudly.
   ✅ B. A motor is whirring consistently.
      C. This is the last event, no immediate event after.

🎯 Correct Answer: B - A motor is whirring consistently.

🤖 Audio Flamingo 3 Prediction: ❌
   Predicted: C - This is the last event, no immediate event after.
   Incorrect

🔊 Audio: 100481.mp3




Example 7: 100481.mp3__1

📝 Question:
   What happens immediately after this event first appears: "A motor is whirring consistently."?

📋 Options:
   ✅ A. A bird chirps loudly.
      B. This is the last event, no immediate event after.
      C. Birds chirping in the distance.

🎯 Correct Answer: A - A bird chirps loudly.

🤖 Audio Flamingo 3 Prediction: ❌
   Predicted: B - This is the last event, no immediate event after.
   Incorrect

🔊 Audio: 100481.mp3




Example 8: 100481.mp3__2

📝 Question:
   What happens immediately after this event first appears: "A bird chirps loudly."?

📋 Options:
      A. Birds chirping in the distance.
   ✅ B. This is the last event, no immediate event after.
      C. A motor is whirring consistently.

🎯 Correct Answer: B - This is the last event, no immediate event after.

🤖 Audio Flamingo 3 Prediction: ❌
   Predicted: C - A motor is whirring consistently.
   Incorrect

🔊 Audio: 100481.mp3




Example 9: 100489.mp3__0

📝 Question:
   What happens immediately after this event first appears: "A bell rings repeatedly in the background."?

📋 Options:
      A. A scratching noise.
      B. A whistling sound is heard in the background.
      C. This is the last event, no immediate event after.
   ✅ D. Low-pitched engine running.

🎯 Correct Answer: D - Low-pitched engine running.

🤖 Audio Flamingo 3 Prediction: ❌
   Predicted: C - This is the last event, no immediate event after.
   Incorrect

🔊 Audio: 100489.mp3




Example 10: 100489.mp3__1

📝 Question:
   What happens immediately after this event first appears: "Low-pitched engine running."?

📋 Options:
   ✅ A. A scratching noise.
      B. A whistling sound is heard in the background.
      C. A bell rings repeatedly in the background.
      D. This is the last event, no immediate event after.

🎯 Correct Answer: A - A scratching noise.

🤖 Audio Flamingo 3 Prediction: ❌
   Predicted: D - This is the last event, no immediate event after.
   Incorrect

🔊 Audio: 100489.mp3
